In [0]:
'''
Author : Parasharaaaaa
'''

'\nAuthor : Parasharaaaaa\n'

In [0]:
from __future__ import absolute_import, division, print_function

In [0]:
import tensorflow as tf

tf.enable_eager_execution()
tf.__version__

'1.13.1'

In [0]:
from tensorflow.keras.layers import Bidirectional,CuDNNGRU,CuDNNLSTM,GRU,LSTM,concatenate,Dense,Input,Embedding
from tensorflow.keras.layers import  multiply, Flatten
from tensorflow.keras.layers import Lambda, Masking
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from hyperopt import Trials, STATUS_OK, tpe,  fmin, tpe, hp, STATUS_OK, Trials
import importlib
from keras import backend as K

Using TensorFlow backend.


In [0]:
!pip install h5py

!apt-get install -y libhdf5-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libhdf5-dev is already the newest version (1.10.0-patch1+docs-4).
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.


In [0]:
import h5py

In [0]:
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle

## Download and prepare the MS-COCO dataset

In [0]:
annotation_zip = tf.keras.utils.get_file('captions.zip', 
                                          cache_subdir=os.path.abspath('.'),
                                          origin = 'http://images.cocodataset.org/annotations/annotations_trainval2014.zip',
                                          extract = True)
annotation_file = os.path.dirname(annotation_zip)+'/annotations/captions_train2014.json'

name_of_zip = 'train2014.zip'
if not os.path.exists(os.path.abspath('.') + '/' + name_of_zip):
  image_zip = tf.keras.utils.get_file(name_of_zip, 
                                      cache_subdir=os.path.abspath('.'),
                                      origin = 'http://images.cocodataset.org/zips/train2014.zip',
                                      extract = True)
  PATH = os.path.dirname(image_zip)+'/train2014/'
else:
  PATH = os.path.abspath('.')+'/train2014/'

13510574080/13510573713 [==============================] - 363s 0us/step


## Optionally, limit the size of the training set for faster training

In [0]:
# read the json file
with open(annotation_file, 'r') as f:
    annotations = json.load(f)

# storing the captions and the image name in vectors
all_captions = []# only has the captions
all_img_name_vector = []# only has the paths

for annot in annotations['annotations']:
    caption = '<start> ' + annot['caption'] + ' <end>'
    image_id = annot['image_id']
    full_coco_image_path = PATH + 'COCO_train2014_' + '%012d.jpg' % (image_id)
    
    all_img_name_vector.append(full_coco_image_path)
    all_captions.append(caption)

# shuffling the captions and image_names together
# setting a random state
train_captions, img_name_vector = shuffle(all_captions,
                                          all_img_name_vector,
                                          random_state=1)

# selecting the first 10000 captions from the shuffled set
num_examples = 2000
train_captions = train_captions[:num_examples]
img_name_vector = img_name_vector[:num_examples]

In [0]:
img_name_vector[:5]

['/content/train2014/COCO_train2014_000000324909.jpg',
 '/content/train2014/COCO_train2014_000000511972.jpg',
 '/content/train2014/COCO_train2014_000000508809.jpg',
 '/content/train2014/COCO_train2014_000000270497.jpg',
 '/content/train2014/COCO_train2014_000000008014.jpg']

## Preprocess the images using InceptionV3

In [0]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

## Initialize InceptionV3 and load the pretrained Imagenet weights

In [0]:
image_model = tf.keras.applications.InceptionV3(include_top=False, 
                                                weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

Instructions for updating:
Colocations handled automatically by placer.
87916544/87910968 [==============================] - 3s 0us/step


**Code to cache the inceptionV3  tensors**

---



In [0]:
'''experimental stuff!!'''
'''
https://github.com/tensorflow/tensorflow/issues/25731
https://github.com/tensorflow/tensorflow/issues/25808
'''


# getting the unique images
encode_train = sorted(set(img_name_vector))

# feel free to change the batch_size according to your system configuration
# a generator which yields 16 images at one shot along with its path
image_dataset = tf.data.Dataset.from_tensor_slices(
                                encode_train).map(load_image).batch(16)

for img, path in image_dataset:
  #image_features_extract_model is the InceptionV3 stack which gives out tensors in the end 
  batch_features = image_features_extract_model(img)
  batch_features = tf.reshape(batch_features, 
                              (batch_features.shape[0], -1, batch_features.shape[3]))
  
  #   #for the eager tensor apparently wrapping it using np.array works which you can use instead of .numpy()
  #   batch_features= np.array(batch_features)
  #   print(batch_features)
  #this is the workaround for the path.numpy
  #   utfdecoder = lambda t: t.decode('utf-8')
  #   utffunc = np.vectorize(utfdecoder)
  #   path_of_features = utffunc(path.numpy())  

  for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    np.save(path_of_feature, bf.numpy())
  


In [0]:
# loading the numpy files 
def map_func(cap,img_name):
  #caching the features stored in the .npy files
  image_tensor = np.load(img_name.numpy().decode('utf-8') +'.npy')
  return cap.numpy(), image_tensor

#old code for non caching of features
#   batch_features = image_features_extract_model(tf.expand_dims(load_image(img_name)[0], axis = 0))
#   image_tensor = tf.reshape(batch_features, (batch_features.shape[0], -1, batch_features.shape[3]))
#   image_tensor = tf.squeeze(image_tensor)

  
          

## Preprocess and tokenize the captions

In [0]:
# This will find the maximum length of any caption in our dataset
def calc_max_length(tensor):
    return max(len(t) for t in tensor)

In [0]:
# The steps above is a general process of dealing with text processing
# choosing the top 5000 words from the vocabulary
top_k = 5000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k, 
                                                  oov_token="<unk>", 
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(train_captions)
train_seqs = tokenizer.texts_to_sequences(train_captions)

In [0]:
tokenizer.word_index['<pad>'] = 0

In [0]:
# creating the tokenized vectors
train_seqs = tokenizer.texts_to_sequences(train_captions)

In [0]:
# padding each vector to the max_length of the captions
# if the max_length parameter is not provided, pad_sequences calculates that automatically
cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

In [0]:
cap_vector.shape


(2000, 40)

In [0]:
# calculating the max_length 
# used to store the attention weights
max_length = calc_max_length(train_seqs)
max_length

40

## Split the data into training and testing

In [0]:
# Create training and validation sets using 80-20 split
img_name_train, img_name_val, cap_train, cap_val = train_test_split(img_name_vector, 
                                                                    cap_vector, 
                                                                    test_size=0.2, 
                                                                    random_state=0)

**MODEL HYPERPARAMETERS AND CONFIG PARAMS**

In [0]:
# feel free to change these parameters according to your system's configuration

BATCH_SIZE = 8
BUFFER_SIZE = 1000
embedding_dim = 256
units = 512
vocab_size = len(tokenizer.word_index)
# shape of the vector extracted from InceptionV3 is (64, 2048)
# these two variables represent that
features_shape = 2048
attention_features_shape = 64

model_config = {
    'save_dir': '../saved_models',
    'dim_cnn': 4096,
    'optimizer': 'adam',
    'batch_size': BATCH_SIZE,
    'epoch': 2,
    'output_dim': 1024,
    'dim_word': 300,
    'lrate': 0.05,
    #'lrate': 0.0005,
    'max_cap_length' : max_length,
    'margin': 0.05, #alpha
    'glove.path': '../data/glove.6B.300d.txt'
}

**CREATING THE DATASET GENERATOR USING tf.data.Dataset**

In [0]:
#create a simple datagenerator using the coco image features and the captions
def create_data_generator():
  dataset = tf.data.Dataset.from_tensor_slices((cap_train,img_name_train))

  # using map to load the numpy files in parallel
  # NOTE: Be sure to set num_parallel_calls to the number of CPU cores you have
  # https://www.tensorflow.org/api_docs/python/tf/py_func
  dataset = dataset.map(lambda item1, item2: tf.py_function(
            map_func, [item1, item2], [tf.int32, tf.float32]), num_parallel_calls=8)

  # shuffling and batching
  dataset = dataset.shuffle(BUFFER_SIZE)
  # https://www.tensorflow.org/api_docs/python/tf/contrib/data/batch_and_drop_remainder
  dataset = dataset.batch(BATCH_SIZE)
  dataset = dataset.repeat()
  dataset = dataset.prefetch(1)

  return dataset
  
  
  

In [0]:
#using the above data_generator to yield training data to be fitted to the model at training time
def train_generator():
  train_iter = create_data_generator().make_one_shot_iterator()
  while True:
    caps, imgs = train_iter.get_next()
    batch = [caps.numpy(),imgs.numpy()]
    dummy = np.zeros(shape=(BATCH_SIZE, model_config['output_dim'] * 2))    
    yield batch, dummy
    

def test_generator():
  train_iter = create_data_generator(cap_val, img_name_val).make_one_shot_iterator()
  while True:
    caps, imgs = train_iter.get_next()
    batch = [caps.numpy(),imgs.numpy()]
    dummy = np.zeros(shape=(BATCH_SIZE, model_config['output_dim'] * 2))    
    yield batch, dummy


**TESTING THE DATASET GENERATOR**

In [0]:
iterator = create_data_generator().make_one_shot_iterator()
x,y = next(iterator)
print(x.shape)
print(y.shape)

TypeError: ignored

##Model

In [0]:
#A GRU WRAPPER
def gru(units):
  # If you have a GPU, we recommend using the CuDNNGRU layer (it provides a 
  # significant speedup).
  if tf.test.is_gpu_available():
    return CuDNNGRU(units, 
                    return_sequences=False, 
                    return_state=False, 
                    recurrent_initializer='glorot_uniform')
  else:
    return GRU(units, 
                 return_sequences=False, 
                 return_state=True, 
                 recurrent_activation='sigmoid', 
                 recurrent_initializer='glorot_uniform')

#RELATED TO LOSS FUNCTIONS
def compute_errors(s_emb, im_emb):
    """ Given sentence and image embeddings, compute the error matrix """
    errors = [order_violations(x, y) for x in s_emb for y in im_emb]
    return np.asarray(errors).reshape((len(s_emb), len(im_emb)))

def order_violations(s, im):
    """ 
    Computes the order violations (Equation 2 in the paper) 
    i.e this is the similiarity score -S(c,i)
    """
    
    return np.power(np.linalg.norm(s - im),2)


def l2norm(X):
    """ Compute L2 norm, row-wise """
    norm = tf.reduce_sum(tf.sqrt(tf.pow(X, 2)), 1)
    X /= norm[:, None]
    return X



def contrastive_loss(labels, predict):
    """For a minibatch of sentence and image embeddings, compute the pairwise contrastive loss, refer the original report to understand this better"""
    global model_options
    margin = model_config['margin']
    predict = tf.identity(predict)
    s, im = tf.split(predict, [model_config['output_dim'], model_config['output_dim']], axis = -1)
    print("s=",s)
    print("im=",im)

    im2 = tf.expand_dims(im,axis=0)
    s2 = tf.expand_dims(s,axis=1)
    print("im2=",im2)
    print("s2=",s2)

    errors = tf.reduce_sum(tf.pow(tf.subtract(im2, s2), 2), 2)
    diagonal = tf.diag_part(errors)
    cost_s = tf.maximum(tf.constant(0, dtype=tf.float32), tf.add(tf.subtract(margin, errors), diagonal))
    diagonal2 = tf.reshape(diagonal, [-1, 1])
    print("\n\n\n\n")
    # print("margin.shape = ", margin.shape)
    print("errors.shape = ", errors.shape)
    print("diagonal.shape = ", diagonal.shape)
    print("diagonal2.shape = ", diagonal2.shape)
    print("\n\n\n\n")
    cost_im = tf.maximum(tf.constant(0, dtype=tf.float32), tf.add(tf.subtract(margin, errors), diagonal2))
    cost_tot = tf.add(cost_s, cost_im)
    print("cost_s=",cost_s)
    print("cost_im=",cost_im)
    print("cost_tot before=",cost_tot)
    zeroes = tf.zeros((model_config['batch_size'],), tf.float32)
    print("zeroes=",zeroes)
    cost_tot = tf.linalg.set_diag(cost_tot, zeroes)
    print("cost_tot after=",cost_tot)
    return tf.reduce_sum(cost_tot)


In [0]:
#for evaluating the model, look at this shit carefully!!
def contrastive_loss_cap(labels,predict):
  s = tf.identity(predict)
  pass

def contrastive_loss_img(labels,predict):
  im = tf.identity(predict)
  pass

**MODEL DEFINITION**

In [0]:
#IMAGE SIDE (DECODER SIDE)
print("image model loading...")
image_input = Input(shape=(64, 2048), name='image_input')
X = Flatten()(image_input)
X = Dense(model_config['output_dim'])(X)
emb_image = Lambda(lambda x: l2norm(x))(X)   #the final embedding representation of the image

#CAPTION SIDE (ENCODER SIDE)
print ("Text model loading..")
cap_input = Input(shape=(model_config['max_cap_length'],), dtype='float32', name='cap_input')
X = Masking(mask_value=0,input_shape=(model_config['max_cap_length'], model_config['output_dim']))(X)
# from scratch
X = Embedding(output_dim=model_config['dim_word'], input_dim=len(tokenizer.word_index), input_length=model_config['max_cap_length'])(cap_input)
# pretrained GloVe
# X = Embedding(output_dim=model_config['dim_word'], input_dim=len(worddict)+2, input_length=model_config['max_cap_length'], weights=[embedding_matrix], trainable=True)(cap_input)
X = gru(model_config['output_dim'])(X)
emb_cap = Lambda(lambda x: l2norm(x))(X)   #the final embedding representation of the catption

#CONCATENATING BOTH MODAL REPRESENTATIONS
print ("loading the joined model")
merged = concatenate([emb_cap, emb_image])

#MODEL INPUT/OUTPUT DEFINTION
model = Model(inputs=[cap_input, image_input], outputs=[merged])

#MODEL COMPILING STEP
print ("compiling the model")
model.compile(optimizer=model_config['optimizer'], loss=contrastive_loss)

image model loading...
Text model loading..
loading the joined model
compiling the model
s= Tensor("loss/concatenate_2_loss/split:0", shape=(?, 1024), dtype=float32)
im= Tensor("loss/concatenate_2_loss/split:1", shape=(?, 1024), dtype=float32)
im2= Tensor("loss/concatenate_2_loss/ExpandDims:0", shape=(1, ?, 1024), dtype=float32)
s2= Tensor("loss/concatenate_2_loss/ExpandDims_1:0", shape=(?, 1, 1024), dtype=float32)





errors.shape =  (?, ?)
diagonal.shape =  (?,)
diagonal2.shape =  (?, 1)





cost_s= Tensor("loss/concatenate_2_loss/Maximum:0", shape=(?, ?), dtype=float32)
cost_im= Tensor("loss/concatenate_2_loss/Maximum_1:0", shape=(?, ?), dtype=float32)
cost_tot before= Tensor("loss/concatenate_2_loss/Add_2:0", shape=(?, ?), dtype=float32)
zeroes= Tensor("loss/concatenate_2_loss/zeros:0", shape=(8,), dtype=float32)
cost_tot after= Tensor("loss/concatenate_2_loss/MatrixSetDiag:0", shape=(?, ?), dtype=float32)


**MODEL ARCHITECTURE SUMMARY**

In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
cap_input (InputLayer)          (None, 40)           0                                            
__________________________________________________________________________________________________
image_input (InputLayer)        (None, 64, 2048)     0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 40, 300)      692400      cap_input[0][0]                  
__________________________________________________________________________________________________
flatten (Flatten)               (None, 131072)       0           image_input[0][0]                
__________________________________________________________________________________________________
cu_dnngru 

**LOAD THE SAVED MODEL WEIGHTS BEFORE RETRAINING**

In [0]:
#Setup between colab and drive and fuse or something!!
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131322 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
#load from model.json and try stuff...
from tensorflow.keras.models import model_from_json
# load json and create model
with open('drive/DLSIR_model_weights/model.json', 'r') as json_file:
  loaded_model_json = json_file.read()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("drive/DLSIR_model_weights/model_weights.h5")
model = loaded_model
print("Loaded model from disk")

Loaded model from disk


In [0]:
#recompile this loaded model
model.compile(optimizer=model_config['optimizer'], loss=contrastive_loss)

s= Tensor("loss_2/concatenate_2_loss/split:0", shape=(?, 1024), dtype=float32)
im= Tensor("loss_2/concatenate_2_loss/split:1", shape=(?, 1024), dtype=float32)
im2= Tensor("loss_2/concatenate_2_loss/ExpandDims:0", shape=(1, ?, 1024), dtype=float32)
s2= Tensor("loss_2/concatenate_2_loss/ExpandDims_1:0", shape=(?, 1, 1024), dtype=float32)





errors.shape =  (?, ?)
diagonal.shape =  (?,)
diagonal2.shape =  (?, 1)





cost_s= Tensor("loss_2/concatenate_2_loss/Maximum:0", shape=(?, ?), dtype=float32)
cost_im= Tensor("loss_2/concatenate_2_loss/Maximum_1:0", shape=(?, ?), dtype=float32)
cost_tot before= Tensor("loss_2/concatenate_2_loss/Add_2:0", shape=(?, ?), dtype=float32)
zeroes= Tensor("loss_2/concatenate_2_loss/zeros:0", shape=(8,), dtype=float32)
cost_tot after= Tensor("loss_2/concatenate_2_loss/MatrixSetDiag:0", shape=(?, ?), dtype=float32)


**TIME TO FIT THE DATA AND TRAIN THE MODEL!**

In [0]:
# #model.fit
# #to do write code to save checkpoints of model weights
# train_hist = model.fit(train_generator(), batch_size = BATCH_SIZE, steps_per_epoch=(len(cap_train) // BATCH_SIZE),verbose=1,epochs = 2) 

In [0]:
#fitgenerator
train_gen = train_generator()
train_hist = model.fit_generator(train_gen, steps_per_epoch=(len(cap_train) // BATCH_SIZE),verbose=1,epochs = 10)

Epoch 1/10
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


200/200 [==============================] - 39s 194ms/step - loss: 5.6000
Epoch 2/10
200/200 [==============================] - 36s 179ms/step - loss: 5.6000
Epoch 3/10
200/200 [==============================] - 36s 179ms/step - loss: 5.6000
Epoch 4/10
200/200 [==============================] - 36s 179ms/step - loss: 5.5993
Epoch 5/10
200/200 [==============================] - 36s 179ms/step - loss: 5.5201
Epoch 6/10
200/200 [==============================] - 36s 179ms/step - loss: 5.3565
Epoch 7/10
200/200 [==============================] - 36s 179ms/step - loss: 5.1950
Epoch 8/10
200/200 [==============================] - 36s 179ms/step - loss: 5.0338
Epoch 9/10
200/200 [==============================] - 36s 179ms/step - loss: 4.8875
Epoch 10/10
200/200 [==============================] - 36s 179ms/step - loss: 4.7945


**SAVE THE WEIGHTS INTO DRIVE AND BACK**

In [0]:
#save the weights
model_json = model.to_json()
with open("drive/DLSIR_model_weights/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("drive/DLSIR_model_weights/model_weights.h5")
print("Saved model to drive")

FileNotFoundError: ignored

**LOOK AT THE MODEL WEIGHTS AND SEE STUFF**



In [0]:
 weights = model.get_weights()

In [0]:
for i in range(len(weights)):
  print(weights[i].shape)


(2308, 300)
(300, 3072)
(1024, 3072)
(6144,)
(131072, 1024)
(1024,)


**EVALUATION HELPER FUNCTIONS**

In [0]:
def input2image(c2i):
    """
    Input caption->Images (Image Search)
    c2i: (1, N) vector of caption to image errors
    returns indices of 10 most likely pictures
    """
    
    inds = np.zeros(c2i.shape, dtype='int32')
    
    for i in range(c2i.shape[0]):
        inds[i,:] = np.argsort(c2i[i,:])
    
    return inds[:,0:10]

def t2i(c2i):
    """
    Text->Images (Image Search)
    c2i: (5N, N) matrix of caption to image errors
    vis_details: if true, return a dictionary for ROC visualization purposes
    """

    ranks = np.zeros(c2i.shape[0])

    for i in range(len(ranks)):
        d_i = c2i[i]
        inds = np.argsort(d_i)

        rank = np.where(inds == int(i/5))[0][0]
        ranks[i] = rank

        def image_dict(k):
            return {'id': k, 'score': float(d_i[k])}

    r10 = 100.0 * len(np.where(ranks < 10)[0]) / len(ranks)
    meanr = ranks.mean() + 1

    stats = map(float, [r10, meanr])

    return stats



def i2t(c2i):
    """
    Text->Images (Image Search)
    c2i: (5N, N) matrix of caption to image errors
    """

    ranks = np.zeros(c2i.shape[1])

    for i in range(len(ranks)):
        d_i = c2i[:, i]
        inds = np.argsort(d_i)

        rank = np.where((inds/5).astype(int) == i)[0][0]
        ranks[i] = rank

    # Compute metrics
    r10 = 100.0 * len(np.where(ranks < 10)[0]) / len(ranks)
    meanr = ranks.mean() + 1
    return map(float, [r10, meanr])


In [0]:
def eval_model():
  #have to look at this shit carefully and look 
  weights = model.get_weights()

  emb_w = weights[0]
  im_w = weights[4]
  im_b = weights[5]
  gru_weights = weights[1:4]

  test_model_im = Model(inputs=image_input, outputs=emb_image)
  test_model_im.set_weights([im_w, im_b])
  test_model_im.compile(optimizer='adam', loss="mse")#contrastive_loss)
  test_model_cap = Model(inputs=cap_input, outputs=emb_cap)
  test_model_cap.set_weights([emb_w]+ gru_weights)
  test_model_cap.compile(optimizer='adam', loss="mse")#contrastive_loss)

  #have to look at this shit!
  test_cap, test_im = test_iter.all()

  pred_cap = test_model_cap.predict(test_cap)
  pred_im = test_model_im.predict(test_im)
  test_errs = compute_errors(pred_cap, pred_im)

  r10_c, rmean_c = t2i(test_errs)
  r10_i, rmean_i = i2t(test_errs)
  print ("Image to text: %.1f %.1f" % (r10_i, rmean_i))
  print ("Text to image: %.1f %.1f" % (r10_c, rmean_c))

In [0]:
eval_model()

Instructions for updating:
Use tf.cast instead.


NameError: ignored